#### Task 1

Set up cssandra cluster of three nodes
* https://hub.docker.com/_/cassandra 
* https://gokhanatil.com/2018/02/build-a-cassandra-cluster-on-docker.html 
* https://www.jamescoyle.net/how-to/2448-create-a-simple-cassandra-cluster-with-3-nodes
* https://www.digitalocean.com/community/tutorials/how-to-run-a-multi-node-cluster-database-with-cassandra-on-ubuntu-14-04


Cassandra cluster is configured with docker compose.

Check file *docker-compose.yml* in this repo.

#### Task 2

Check the cluster configuration with *nodetool status*

#### Task 3

Using *cqlsh* create three Keyspaces
* https://www.tutorialspoint.com/cassandra/cassandra_create_keyspace.htm
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_keyspace_r.html


#### Task 4

Create tables in every Keyspace
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/create_table_r.html
* https://www.tutorialspoint.com/cassandra/cassandra_create_table.htm


#### Task 5

Try to read and write to/from different nodes

#### Task 6

Insert data to tables and check their distribution between nodes (for each Keyspace use *nodetool status*)
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/insert_r.html
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/select_r.html 
* https://www.tutorialspoint.com/cassandra/cassandra_create_data.htm
* https://www.tutorialspoint.com/cassandra/cassandra_read_data.htm


#### Task 7

For each record from each keyspace show nodes where they are stored
* https://docs.datastax.com/en/dse/5.1/dse-admin/datastax_enterprise/tools/nodetool/toolsGetEndPoints.html

#### Task 8

Switch of one of the nodes. For each keyspace figure out a consistency level for reading and writing, which of them guranty strong consistency
* https://docs.datastax.com/en/cql/3.1/cql/cql_reference/consistency_r.html


#### Task 9

Disable networking leaving all nodes switched on.

#### Task 10

For Keyspace with replication factor 3 make consistency level 1. Write one record (with same primary key but different values to each node.

#### Task 11

Turn on networking and figure out whicj value is desided to be agreed by cluster.

#### Task 12

Check the behavour of lightweight transactions for previous tasks in partitioned and consolidated cluster.
* https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertLWT.html 
